<a href="https://colab.research.google.com/github/dinethjanitha/London-House-Price-Prediction/blob/main/LondonHousePredictApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 98.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 103.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.4 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
import streamlit as st
import joblib
import pandas as pd # Import pandas to create DataFrame for prediction

st.set_page_config(layout="wide", page_title="London House Price Predictor")

st.write("# London House Price Predict")

col1, col2, col3 = st.columns(3)

bathroom_count = col1.number_input("Enter bathroom count", min_value=0, step=1, value=1)

livingroom_count = col2.number_input("Enter living rooms count", min_value=0, step=1, value=1)

bedroom_count = col3.number_input("Enter bedroom count", min_value=0, step=1, value=2)

floor_area_sqm = col1.number_input("Enter floor area square meters", min_value=0, step=1, value=75)

latitude = col2.number_input("Enter latitude", min_value=-90.0, max_value=90.0, format="%.6f", value=51.5074)

longitude = col3.number_input("Enter longitude", min_value=-180.0, max_value=180.0, format="%.6f", value=0.1278)

sale_month = col1.selectbox("Enter sale month", [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], index=0)

current_year = 2025
years = list(range(current_year - 20, current_year + 1))
sale_year = col2.selectbox("Enter sale year", options=years, index=len(years)-1)

tenure_options = ['Leasehold', 'Freehold', 'Feudal', 'Shared']
tenure = col3.selectbox("Enter tenure", tenure_options, index=0)

energy_rate = col1.selectbox("Enter energy rating", [0 , 1, 2 ,3 ,4 ,5 ,6], index=0)

property_type_options = [
    'Terrace Property', 'Mid Terrace House', 'Converted Flat',
    'Purpose Built Flat', 'Semi-Detached House', 'End Terrace House',
    'Flat/Maisonette', 'Detached House','Bungalow Property', 'Terraced',
    'Mid Terrace Property', 'Detached Bungalow' ,'Semi-Detached Bungalow',
    'End Terrace Property', 'Terraced Bungalow', 'Mid Terrace Bungalow',
    'Detached Property', 'Semi-Detached Property', 'End Terrace Bungalow'
]
propertyType = col2.selectbox("Enter Property type", property_type_options, index=0)

outcode_options = [
    'E1', 'E10', 'E11', 'E13', 'E14', 'E15', 'E17', 'E18', 'E3', 'E4',
    'E5', 'E8', 'EC2Y', 'N10', 'N11', 'N12', 'N13', 'N14', 'N15',
    'N16', 'N17', 'N19', 'N2', 'N21', 'N22', 'N5', 'N6', 'N7', 'N8',
    'N9', 'NW1', 'NW10', 'NW11', 'NW2', 'NW3', 'NW4', 'NW5', 'NW6',
    'NW8', 'SE1', 'SE10', 'SE11', 'SE13', 'SE15', 'SE16', 'SE17',
    'SE18', 'SE2', 'SE20', 'SE21', 'SE23', 'SE24', 'SE25', 'SE26',
    'SE27', 'SE28', 'SE3', 'SE4', 'SE5', 'SE6', 'SE7', 'SE9', 'SW10',
    'SW11', 'SW12', 'SW13', 'SW14', 'SW15', 'SW16', 'SW17', 'SW18',
    'SW19', 'SW1P', 'SW1V', 'SW1X', 'SW2', 'SW3', 'SW4', 'SW5', 'SW6',
    'SW7', 'W10', 'W11', 'W12', 'W13', 'W14', 'W1F', 'W1H', 'W1J',
    'W1W', 'W2', 'W4', 'W6', 'W7', 'W8', 'WC1H', 'WC1R', 'WC1X',
    'WC2B', 'WC2H', 'E6', 'E7', 'N1', 'N18', 'N20', 'N3', 'N4', 'NW9',
    'SE12', 'SE19', 'SE22', 'SE8', 'SW1H', 'SW20', 'SW8', 'SW9', 'W1K',
    'W1T', 'W5', 'WC1A', 'WC1B', 'WC1E', 'WC2E', 'E12', 'E2', 'E9',
    'NW7', 'SE14', 'SW1W', 'W1D', 'W1G', 'W1U', 'W3', 'W9', 'WC1N',
    'EC4R', 'SW1E', 'EC1R', 'EC4A', 'SW1A', 'W1S', 'E16', 'WC2N',
    'EC4V', 'E1W', 'EC1N', 'W1B', 'SW1Y', 'EC2A', 'EC3V', 'EC1A',
    'EC1Y', 'EC4Y', 'EC1V', 'EC3A', 'EC3N', 'EC1M', 'EC4M', 'WC2R',
    'EC2M', 'EC3R', 'WC2A', 'WC1V', 'EC2V', 'EC2R', 'W1C'
]
outcode = col3.selectbox("Enter outcode", outcode_options, index=0)

tenure_map = {option: i for i, option in enumerate(tenure_options)}

property_type_map = {option: i for i, option in enumerate(property_type_options)}

outcode_map = {option: i for i, option in enumerate(outcode_options)}

@st.cache_resource
def load_model():
    try:
        model = joblib.load('/content/drive/MyDrive/Assignment/StreamLit/londonhousepredict.pkl')
        return model
    except FileNotFoundError:
        st.error("Model file not found. Please ensure 'londonhousepredict.pkl' is in the specified path.")
        return None
    except Exception as e:
        st.error(f"Error loading model: {e}")
        return None

model = load_model()

st.markdown("---")
if st.button("Predict House Price"):
    if model is not None:
        numerical_tenure = tenure_map[tenure]
        numerical_property_type = property_type_map[propertyType]
        numerical_outcode = outcode_map[outcode]


        input_data = pd.DataFrame([[
            numerical_outcode,
            latitude,
            longitude,
            bathroom_count,
            floor_area_sqm,
            livingroom_count,
            numerical_tenure,
            numerical_property_type,
            energy_rate,
            sale_year
        ]], columns=[
            'outcode', 'latitude', 'longitude', 'bathrooms', 'floorAreaSqM',
            'livingRooms', 'tenure', 'propertyType', 'currentEnergyRating',
            'sale_year'
        ])

        try:
            # Make prediction
            predicted_price = model.predict(input_data)[0]
            st.markdown(f"### Predicted Price: £ {predicted_price:,.2f}")
        except Exception as e:
            st.error(f"Error during prediction: {e}")
            st.write("Please ensure the model expects these exact features and their order.")
    else:
        st.warning("Model could not be loaded. Cannot make prediction.")



Overwriting app.py


In [ ]:
%%writefile app.py
import streamlit as st
import joblib
import pandas as pd

# Page configuration
st.set_page_config(
    layout="wide",
    page_title="London House Price Predictor",
    page_icon="🏠"
)

# Custom CSS styling
st.markdown("""
    <style>
        .main {
            background-color: #f5f5f5;
        }
        .block-container {
            padding-top: 2rem;
        }
        .title-text {
            font-size: 3rem;
            color: #2c3e50;
            font-weight: bold;
        }
        .predict-button > button {
            background-color: #27ae60;
            color: white;
            font-size: 1.1rem;
            border-radius: 10px;
            padding: 0.5rem 1rem;
        }
        .predict-button > button:hover {
            background-color: #219150;
        }
        .price-box {
            background-color: #dff0d8;
            color: #2c3e50;
            padding: 1rem;
            border-radius: 15px;
            font-size: 1.5rem;
            font-weight: bold;
            text-align: center;
        }
    </style>
""", unsafe_allow_html=True)

st.markdown('<div class="title-text">🏠 London House Price Predictor</div>', unsafe_allow_html=True)
st.markdown("---")

col1, col2, col3 = st.columns(3)

with col1:
    bathroom_count = st.number_input("🛁 Bathroom count", min_value=0, step=1, value=1)
    floor_area_sqm = st.number_input("📐 Floor area (sq meters)", min_value=0, step=1, value=75)
    sale_month = st.selectbox("📅 Sale month", list(range(1, 13)), index=0)
    energy_rate = st.selectbox("⚡ Energy rating", [0, 1, 2, 3, 4, 5, 6], index=0)

with col2:
    livingroom_count = st.number_input("🛋️ Living rooms count", min_value=0, step=1, value=1)
    latitude = st.number_input("🌍 Latitude", min_value=-90.0, max_value=90.0, format="%.6f", value=51.5074)
    current_year = 2025
    years = list(range(current_year - 20, current_year + 1))
    sale_year = st.selectbox("📆 Sale year", options=years, index=len(years)-1)
    propertyType = st.selectbox("🏘️ Property type", [
        'Terrace Property', 'Mid Terrace House', 'Converted Flat',
        'Purpose Built Flat', 'Semi-Detached House', 'End Terrace House',
        'Flat/Maisonette', 'Detached House','Bungalow Property', 'Terraced',
        'Mid Terrace Property', 'Detached Bungalow' ,'Semi-Detached Bungalow',
        'End Terrace Property', 'Terraced Bungalow', 'Mid Terrace Bungalow',
        'Detached Property', 'Semi-Detached Property', 'End Terrace Bungalow'
    ], index=0)

with col3:
    bedroom_count = st.number_input("🛏️ Bedroom count", min_value=0, step=1, value=2)
    longitude = st.number_input("🌍 Longitude", min_value=-180.0, max_value=180.0, format="%.6f", value=0.1278)
    tenure = st.selectbox("📜 Tenure type", ['Leasehold', 'Freehold', 'Feudal', 'Shared'], index=0)
    outcode = st.selectbox("📮 Outcode", [
        'E1', 'E10', 'E11', 'E13', 'E14', 'E15', 'E17', 'E18', 'E3', 'E4',
        'E5', 'E8', 'EC2Y', 'N10', 'N11', 'N12', 'N13', 'N14', 'N15', 'N16',
        'N17', 'N19', 'N2', 'N21', 'N22', 'N5', 'N6', 'N7', 'N8', 'N9', 'NW1',
        'NW10', 'NW11', 'NW2', 'NW3', 'NW4', 'NW5', 'NW6', 'NW8', 'SE1', 'SE10',
        'SE11', 'SE13', 'SE15', 'SE16', 'SE17', 'SE18', 'SE2', 'SE20', 'SE21',
        'SE23', 'SE24', 'SE25', 'SE26', 'SE27', 'SE28', 'SE3', 'SE4', 'SE5',
        'SE6', 'SE7', 'SE9', 'SW10', 'SW11', 'SW12', 'SW13', 'SW14', 'SW15',
        'SW16', 'SW17', 'SW18', 'SW19', 'SW1P', 'SW1V', 'SW1X', 'SW2', 'SW3',
        'SW4', 'SW5', 'SW6', 'SW7', 'W10', 'W11', 'W12', 'W13', 'W14', 'W1F',
        'W1H', 'W1J', 'W1W', 'W2', 'W4', 'W6', 'W7', 'W8', 'WC1H', 'WC1R', 'WC1X',
        'WC2B', 'WC2H', 'E6', 'E7', 'N1', 'N18', 'N20', 'N3', 'N4', 'NW9',
        'SE12', 'SE19', 'SE22', 'SE8', 'SW1H', 'SW20', 'SW8', 'SW9', 'W1K',
        'W1T', 'W5', 'WC1A', 'WC1B', 'WC1E', 'WC2E', 'E12', 'E2', 'E9',
        'NW7', 'SE14', 'SW1W', 'W1D', 'W1G', 'W1U', 'W3', 'W9', 'WC1N',
        'EC4R', 'SW1E', 'EC1R', 'EC4A', 'SW1A', 'W1S', 'E16', 'WC2N',
        'EC4V', 'E1W', 'EC1N', 'W1B', 'SW1Y', 'EC2A', 'EC3V', 'EC1A',
        'EC1Y', 'EC4Y', 'EC1V', 'EC3A', 'EC3N', 'EC1M', 'EC4M', 'WC2R',
        'EC2M', 'EC3R', 'WC2A', 'WC1V', 'EC2V', 'EC2R', 'W1C'
    ], index=0)

tenure_map = {option: i for i, option in enumerate(['Leasehold', 'Freehold', 'Feudal', 'Shared'])}
property_type_map = {option: i for i, option in enumerate([
    'Terrace Property', 'Mid Terrace House', 'Converted Flat',
    'Purpose Built Flat', 'Semi-Detached House', 'End Terrace House',
    'Flat/Maisonette', 'Detached House','Bungalow Property', 'Terraced',
    'Mid Terrace Property', 'Detached Bungalow' ,'Semi-Detached Bungalow',
    'End Terrace Property', 'Terraced Bungalow', 'Mid Terrace Bungalow',
    'Detached Property', 'Semi-Detached Property', 'End Terrace Bungalow'
])}
outcode_map = {option: i for i, option in enumerate(outcode_options)}

@st.cache_resource
def load_model():
    try:
        model = joblib.load('/content/drive/MyDrive/Assignment/StreamLit/londonhousepredict.pkl')
        return model
    except FileNotFoundError:
        st.error("Model file not found.")
        return None
    except Exception as e:
        st.error(f"Error loading model: {e}")
        return None

model = load_model()

st.markdown("---")
with st.container():
    col_center = st.columns([1, 2, 1])[1]
    with col_center:
        if st.button("💰 Predict Price", key="predict", help="Click to predict price", type="primary"):
            if model:
                try:
                    input_data = pd.DataFrame([[
                        outcode_map[outcode],
                        latitude,
                        longitude,
                        bathroom_count,
                        floor_area_sqm,
                        livingroom_count,
                        tenure_map[tenure],
                        property_type_map[propertyType],
                        energy_rate,
                        sale_year
                    ]], columns=[
                        'outcode', 'latitude', 'longitude', 'bathrooms', 'floorAreaSqM',
                        'livingRooms', 'tenure', 'propertyType', 'currentEnergyRating',
                        'sale_year'
                    ])

                    predicted_price = model.predict(input_data)[0]
                    st.markdown(f'<div class="price-box">💷 Predicted Price: £ {predicted_price:,.2f}</div>', unsafe_allow_html=True)

                except Exception as e:
                    st.error(f"Prediction error: {e}")
                    st.info("Ensure the model expects these exact features.")
            else:
                st.warning("Model not loaded. Prediction unavailable.")


Overwriting app.py


In [ ]:
!npm install localtunnel

⠙⠹⠸⠼⠴⠦⠧
up to date, audited 23 packages in 1s
⠧
⠧3 packages are looking for funding
⠧  run `npm fund` for details
⠧
2 high severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠇

In [ ]:
!streamlit run /content/app.py &>/content/logs.txt &

In [ ]:
!npx localtunnel --port 8501

⠙your url is: https://heavy-webs-flow.loca.lt
^C
